In [15]:
from kan import *
from sklearn.neural_network import MLPRegressor

## 生成模拟数据

In [16]:
# 样本量
m = 500
# 协变量特征纬度
n1 = 10
# 观测矩阵维度
n2 = 5
# 协变量矩阵的生成
x = torch.randn(m,n1)
# 模型参数生成
w = torch.normal(0,1,(n1,n2))
#观测矩阵非缺失概率计算
theta = torch.sigmoid(x@w)
#根据概率生成对应的示性矩阵
M = torch.zeros((m,n2))
for i in range(m):
    for j in range(n2):
        M[i,j] = 1 if np.random.uniform(0,1) <= theta[i,j] else 0

data = {'train_input': x[:-100,:],"train_label":M[:-100,:],"test_input": x[-100:,:],"test_label":M[-100:,:]}

## 生成MLP模型以及KAN模型

In [14]:

hidden_size = 5
kan_model = KAN(width=[n1,hidden_size,n2], grid=5, k=3, seed=0)
mlp_model = MLPRegressor(hidden_layer_sizes=hidden_size, activation='sigmoid', solver='adam')
kan_model.train(data, opt="LBFGS", steps=20, lamb=0.01, lamb_entropy=10.);

description:   0%|                                                           | 0/20 [00:00<?, ?it/s]

train loss: 4.83e-01 | test loss: 4.91e-01 | reg: 4.90e+01 : 100%|██| 20/20 [00:09<00:00,  2.03it/s]


In [9]:
kan_model

KAN(
  (biases): ModuleList(
    (0-1): 2 x Linear(in_features=5, out_features=1, bias=False)
  )
  (act_fun): ModuleList(
    (0-1): 2 x KANLayer(
      (base_fun): SiLU()
    )
  )
  (base_fun): SiLU()
  (symbolic_fun): ModuleList(
    (0-1): 2 x Symbolic_KANLayer()
  )
)

In [17]:
f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
dataset = create_dataset(f, n_var=2)
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [20]:
# plot KAN at initialization
model = KAN(width=[2,5,1], grid=5, k=3, seed=0)
model(dataset['train_input']);
model.plot(beta=100)

OSError: [Errno 30] Read-only file system: './figures'